In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[86.0, 95.0, 100.0, 86.0, 40.0, 100.0, 7.0, 35.0, 93.0, 96.0, 7.0, 96.0, 37.0, 37.0, 20.0, 37.0, 96.0, 23.0, 84.0, 100.0, 7.0, 55.0, 100.0, 86.0, 100.0, 7.0, 40.0, 95.0, 4.0, 100.0, 96.0, 32.0, 37.0, 10.0, 93.0, 66.0, 45.0, 96.0, 13.0, 80.0, 55.0, 4.0, 37.0, 37.0, 100.0, 40.0, 40.0, 37.0, 47.0, 55.0, 37.0, 86.0, 55.0, 66.0, 66.0, 13.0, 30.0, 55.0, 47.0, 95.0, 93.0, 37.0, 37.0, 23.0, 16.0, 100.0, 84.0, 30.0, 23.0, 32.0, 86.0, 93.0, 100.0, 93.0, 4.0, 20.0, 37.0, 84.0, 30.0, 40.0, 95.0, 45.0, 23.0, 45.0, 32.0, 95.0, 84.0, 37.0, 32.0, 10.0, 95.0, 13.0, 55.0, 55.0, 13.0, 84.0, 13.0, 84.0, 84.0, 45.0, 37.0, 66.0, 66.0, 55.0, 96.0, 16.0, 84.0, 47.0, 66.0, 7.0, 37.0, 13.0, 52.0, 30.0, 13.0, 32.0, 23.0, 40.0, 84.0, 55.0, 84.0, 55.0, 37.0, 95.0, 13.0, 86.0, 93.0, 52.0, 16.0, 13.0, 45.0, 52.0, 10.0, 86.0, 95.0, 66.0, 80.0, 55.0, 47.0, 96.0, 84.0, 7.0, 95.0, 40.0, 47.0, 37.0, 10.0, 16.0, 37.0, 84.0, 80.0, 7.0, 66.0, 95.0, 66.0, 37.0, 7.0, 16.0, 95.0, 37.0, 13.0, 16.0, 13.0, 13.0, 39.0, 23.0, 100.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [6]:
#要改
RegNetY008 =tf.keras.applications.regnet.RegNetY008(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY008.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY008(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety008 (Functional)     (None, 7, 7, 768)         5524056   
                                                                 
 global_average_pooling2d (G  (None, 768)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 769       
                                                                 
Total params: 5,524,825
Trainable params: 5,494,937
Non-trainable params: 29,888
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY008_loss_4.txt',history.history['loss'])
#要改
np.savetxt('RegNetY008_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Tue Apr 18 08:49:39 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 1096.0455

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


592/592 [==============================] - 98s 139ms/step - loss: 1096.0455 - val_loss: 284.6072
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 136.5705

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


592/592 [==============================] - 117s 198ms/step - loss: 0.3517 - val_loss: 0.1066
Epoch 208/300
592/592 [==============================] - 96s 161ms/step - loss: 0.2019 - val_loss: 0.2164
Epoch 209/300
592/592 [==============================] - 99s 167ms/step - loss: 0.2982 - val_loss: 0.1735
Epoch 210/300
592/592 [==============================] - 100s 168ms/step - loss: 0.3320 - val_loss: 0.1569
Epoch 211/300
592/592 [==============================] - 100s 169ms/step - loss: 0.1927 - val_loss: 0.1558
Epoch 212/300
592/592 [==============================] - 100s 168ms/step - loss: 0.2698 - val_loss: 0.2583
Epoch 213/300
592/592 [==============================] - 99s 168ms/step - loss: 0.2098 - val_loss: 0.2388
Epoch 214/300
592/592 [==============================] - 100s 170ms/step - loss: 0.2525 - val_loss: 0.1343
Epoch 215/300
592/592 [==============================] - 99s 167ms/step - loss: 0.2411 - val_loss: 0.1444
Epoch 216/300
592/592 [==============================] 

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\4\RegNetY008.ckpt\assets


592/592 [==============================] - 120s 203ms/step - loss: 0.2227 - val_loss: 0.1020
Epoch 238/300
592/592 [==============================] - 98s 165ms/step - loss: 0.2173 - val_loss: 0.1037
Epoch 239/300
592/592 [==============================] - 101s 171ms/step - loss: 0.2460 - val_loss: 0.1363
Epoch 240/300
592/592 [==============================] - 100s 169ms/step - loss: 0.1980 - val_loss: 0.1837
Epoch 241/300
592/592 [==============================] - 101s 171ms/step - loss: 0.3416 - val_loss: 0.1147
Epoch 242/300
592/592 [==============================] - 102s 173ms/step - loss: 0.1845 - val_loss: 0.1746
Epoch 243/300
592/592 [==============================] - 101s 171ms/step - loss: 0.2392 - val_loss: 0.1061
Epoch 244/300
592/592 [==============================] - 102s 173ms/step - loss: 0.2622 - val_loss: 0.1570
Epoch 245/300
592/592 [==============================] - 103s 174ms/step - loss: 0.3551 - val_loss: 0.1630
Epoch 246/300
592/592 [=============================